In [20]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from IPython.display import Markdown, display
import ipywidgets as widgets
import pandas as pd
from utils.db import get_column_info, get_pg_engine
from sqlalchemy import text
from sqlalchemy.exc import SQLAlchemyError

load_dotenv()



True

In [3]:
# Initialize OpenAI chat model
llm = ChatOpenAI(
    temperature=0,
    model="gpt-4",
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

In [6]:
# !pipenv run pip install ipywidgets - if don't display data
# Create text inputs for schema and table
schema_input = widgets.Text(
    value='wei',
    description='Schema:',
    disabled=False
)
table_input = widgets.Text(
    value='t_radio_devices_stats_last_status',
    description='Table:',
    disabled=False
)
display(schema_input, table_input)

Text(value='wei', description='Schema:')

Text(value='t_radio_devices_stats_last_status', description='Table:')

In [ ]:
# Get input values
schema = schema_input.value
table = table_input.value

# Define prompt
prompt_template = f"""
Generate a PostgreSQL SQL query that retrieves the name of each column, its data type, and its comment
(description), using system catalog views.

The table is called '{table}' and belongs to the schema '{schema}'.

The query should follow this structure:
SELECT
    a.attname AS column_name,
    format_type(a.atttypid, a.atttypmod) AS data_type,
    col_description(a.attrelid, a.attnum) AS description
FROM
    pg_attribute a
JOIN
    pg_class c ON a.attrelid = c.oid
JOIN
    pg_namespace n ON c.relnamespace = n.oid
WHERE
    c.relname = '{table}'
    AND n.nspname = '{schema}'
    AND a.attnum > 0
    AND NOT a.attisdropped
ORDER BY
    a.attnum;
"""

# Call OpenAI model
response = llm.invoke(prompt_template)
display(Markdown(response.content))

Your query is correct. Here it is again for clarity:

```sql
SELECT
    a.attname AS column_name,
    format_type(a.atttypid, a.atttypmod) AS data_type,
    col_description(a.attrelid, a.attnum) AS description
FROM
    pg_attribute a
JOIN
    pg_class c ON a.attrelid = c.oid
JOIN
    pg_namespace n ON c.relnamespace = n.oid
WHERE
    c.relname = 't_radio_devices_stats_last_status'
    AND n.nspname = 'wei'
    AND a.attnum > 0
    AND NOT a.attisdropped
ORDER BY
    a.attnum;
```

This query will return the name of each column, its data type, and its comment (description) for the table 't_radio_devices_stats_last_status' in the schema 'wei'. It uses the system catalog views `pg_attribute`, `pg_class`, and `pg_namespace` to retrieve this information.


In [10]:
# Execute query on the PostgreSQL database
df = get_column_info(schema, table)
df

,column_name,data_type,description
0,device_sn,character varying(50),None
1,radio_id,integer,None
2,collection_time,timestamp with time zone,None
3,collection_day,timestamp with time zone,None
4,partition_date,date,None
5,up_time,bigint,None
6,bytes_sent,numeric,None
7,bytes_received,numeric,None
8,packets_sent,numeric,None
9,packets_received,numeric,None


In [ ]:
# Textarea widget for user-defined prompt template
prompt_template_widget = widgets.Textarea(
    value=(
        "You are helping document a PostgreSQL table.\n"
        "Write a short and clear description (one sentence) for a column named '{column_name}' of type '{data_type}'.\n"
        "- Do NOT include the column name in the description.\n"
        "- Do NOT include the table name.\n"
        "- Do NOT use quotation marks.\n"
        "Output only the sentence."
    ),
    description='Prompt template:',
    layout=widgets.Layout(width='100%', height='150px')
)

generate_button = widgets.Button(description="Generate Suggestions")
output_box = widgets.Output()

display(widgets.VBox([prompt_template_widget, generate_button, output_box]))

# Function to generate a suggestion for a column
def suggest_comment(column_name, data_type, template):
    prompt = template.format(column_name=column_name, data_type=data_type)
    return llm.invoke(prompt).content.strip().strip('"').strip("'")

# Define the callback when the user clicks the button
def on_generate_clicked(b):
    output_box.clear_output()
    with output_box:
        print("Generating suggestions from LLM...\n")

        # Prepare editable DataFrame
        global df_edit
        df_edit = df.copy()
        df_edit.rename(columns={'description': 'current_description'}, inplace=True)
        df_edit['llm_suggestion'] = None

        template = prompt_template_widget.value

        for i, row in df_edit.iterrows():
            if pd.isna(row['current_description']):
                try:
                    suggestion = suggest_comment(
                        row['column_name'],
                        row['data_type'],
                        template
                    )
                    df_edit.at[i, 'llm_suggestion'] = suggestion
                    print(f"✔ {row['column_name']}: {suggestion}")
                except Exception as e:
                    print(f"❌ Error for {row['column_name']}: {e}")

        print("\n✅ Suggestions stored in `df_edit`")

generate_button.on_click(on_generate_clicked)

In [23]:
df_edit

,column_name,data_type,current_description,llm_suggestion
0,device_sn,character varying(50),None,Stores the unique serial number of up to 50 ch...
1,radio_id,integer,None,Stores the unique identifier for each radio unit.
2,collection_time,timestamp with time zone,None,"Stores the date and time, including the time z..."
3,collection_day,timestamp with time zone,None,"Indicates the date and time, including the tim..."
4,partition_date,date,None,Stores the date when the data was divided into...
5,up_time,bigint,None,Stores the duration for which the system has b...
6,bytes_sent,numeric,None,"Represents the total amount of data, in bytes,..."
7,bytes_received,numeric,None,Represents the total number of bytes received.
8,packets_sent,numeric,None,Represents the number of packets that have bee...
9,packets_received,numeric,None,Stores the number of data packets that have be...


In [24]:
# Store widgets and metadata
input_widgets = []

for _, row in df_edit.iterrows():
    col_label = widgets.HTML(f"<b>{row['column_name']}</b> ({row['data_type']})")
    
    default_text = (
        row['current_description']
        if pd.notna(row['current_description'])
        else row['llm_suggestion'] or ''
    )
    
    comment_input = widgets.Text(
        value=default_text,
        placeholder="Edit description or leave empty to skip"
    )
    
    input_widgets.append((row['column_name'], comment_input, col_label))

# Display inputs
form_rows = [widgets.HBox([label, input_]) for _, input_, label in input_widgets]
display(widgets.VBox(form_rows))

In [25]:
# Collect final descriptions
final_comments = []

for column_name, input_widget, _ in input_widgets:
    comment = input_widget.value.strip()
    if comment:
        final_comments.append({
            "column_name": column_name,
            "comment": comment
        })

final_comments_df = pd.DataFrame(final_comments)
final_comments_df

,column_name,comment
0,device_sn,Stores the unique serial number of up to 50 ch...
1,radio_id,Stores the unique identifier for each radio unit.
2,collection_time,"Stores the date and time, including the time z..."
3,collection_day,"Indicates the date and time, including the tim..."
4,partition_date,Stores the date when the data was divided into...
5,up_time,Stores the duration for which the system has b...
6,bytes_sent,"Represents the total amount of data, in bytes,..."
7,bytes_received,Represents the total number of bytes received.
8,packets_sent,Represents the number of packets that have bee...
9,packets_received,Stores the number of data packets that have be...


In [26]:
# Compose the SQL statements
sql_lines = []

for _, row in final_comments_df.iterrows():
    col = row["column_name"]
    comment = row["comment"].replace("'", "''")  # escape single quotes for SQL
    sql_lines.append(f"COMMENT ON COLUMN {schema}.{table}.{col} IS '{comment}';")

sql_script = "\n".join(sql_lines)

# Display as markdown
markdown_preview = f"```sql\n{sql_script}\n```"
display(Markdown(markdown_preview))

```sql
COMMENT ON COLUMN wei.t_radio_devices_stats_last_status.device_sn IS 'Stores the unique serial number of up to 50 characters for a specific device.';
COMMENT ON COLUMN wei.t_radio_devices_stats_last_status.radio_id IS 'Stores the unique identifier for each radio unit.';
COMMENT ON COLUMN wei.t_radio_devices_stats_last_status.collection_time IS 'Stores the date and time, including the time zone, when the data was collected.';
COMMENT ON COLUMN wei.t_radio_devices_stats_last_status.collection_day IS 'Indicates the date and time, including the time zone, when the data was collected.';
COMMENT ON COLUMN wei.t_radio_devices_stats_last_status.partition_date IS 'Stores the date when the data was divided into subsets.';
COMMENT ON COLUMN wei.t_radio_devices_stats_last_status.up_time IS 'Stores the duration for which the system has been running, measured in milliseconds.';
COMMENT ON COLUMN wei.t_radio_devices_stats_last_status.bytes_sent IS 'Represents the total amount of data, in bytes, that has been transmitted.';
COMMENT ON COLUMN wei.t_radio_devices_stats_last_status.bytes_received IS 'Represents the total number of bytes received.';
COMMENT ON COLUMN wei.t_radio_devices_stats_last_status.packets_sent IS 'Represents the number of packets that have been sent.';
COMMENT ON COLUMN wei.t_radio_devices_stats_last_status.packets_received IS 'Stores the number of data packets that have been received.';
COMMENT ON COLUMN wei.t_radio_devices_stats_last_status.errors_sent IS 'Represents the number of errors that have been sent.';
COMMENT ON COLUMN wei.t_radio_devices_stats_last_status.errors_received IS 'Stores the count of errors received.';
COMMENT ON COLUMN wei.t_radio_devices_stats_last_status.created_at IS 'Stores the date and time when the record was created.';
COMMENT ON COLUMN wei.t_radio_devices_stats_last_status.updated_at IS 'Stores the date and time of the most recent update to the record.';
```

In [ ]:
# Confirm before running the query

confirm_button = widgets.Button(description="Execute SQL")
output_box = widgets.Output()

def on_confirm_clicked(b):
    with output_box:
        output_box.clear_output()
        engine = get_pg_engine()
        success = True

        print("🔍 Executing SQL statements...\n")

        try:
            with engine.begin() as conn:  # uses transaction + autocommit
                for stmt in sql_lines:
                    print(f"→ {stmt}")
                    conn.execute(text(stmt))
            print("\n✅ All comments applied successfully.")
        except SQLAlchemyError as e:
            success = False
            print(f"\n❌ An error occurred:\n{str(e)}")

        if not success:
            print("\nNo changes were committed to the database.")

confirm_button.on_click(on_confirm_clicked)

display(confirm_button, output_box)

Button(description='Execute SQL', style=ButtonStyle())

Output()